In [14]:
!pip install konlpy

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import json
import os
import tqdm

from konlpy.tag import Okt

In [3]:
train=pd.read_csv('./data/train.csv')
test=pd.read_csv('./data/test.csv')
sample_submission=pd.read_csv('./data/sample_submission.csv')

In [6]:
train.head(), test.head(), sample_submission.head()

(   index  제출년도               사업명   사업_부처명 계속과제여부                 내역사업명  \
 0      0  2016          농업기초기반연구    농촌진흥청     신규              농산물안전성연구   
 1      1  2019  이공학학술연구기반구축(R&D)      교육부     신규  지역대학우수과학자지원사업(1년~5년)   
 2      2  2016        중소기업기술혁신개발    중소기업청     신규              혁신기업기술개발   
 3      3  2018     창업성장기술개발(R&D)  중소벤처기업부     신규             창업사업화연계과제   
 4      4  2016       이공학개인기초연구지원      교육부     계속                기본연구지원   
 
                                                  과제명  \
 0                       유전정보를 활용한 새로운 해충 분류군 동정기술 개발   
 1  대장암의 TRAIL 내성 표적 인자 발굴 및 TRAIL 반응 예측 유전자 지도 구축...   
 2  비목질계 셀룰로오스 식물자원을 활용한 기능성 부직포 및 고부가가치 뷰티케어 faci...   
 3                            소화기 암 진단용 분자영상 형광프로브 개발   
 4                        위암환자의 항암제반응예측을 위한 BRCA 발현검사   
 
                                             요약문_연구목표  \
 0                     ○ 새로운 해충분류군의 동정기술 개발 및 유입확산 추적   
 1  최종목표:  TRAIL 감수성 표적 유전자를 발굴하고 내성제어 기전을 연구. 발굴된...   
 2  * 식물계자원 정련 및 최적 신서란 파이버 기초연구 

In [7]:
train.shape, test.shape, sample_submission.shape

((174304, 13), (43576, 12), (43576, 2))

In [8]:
train=train[['요약문_연구내용','label']]
test=test[['요약문_연구내용']]

In [10]:
train.head()

,요약문_연구내용,label
0,(가) 외래 및 돌발해충의 발생조사 및 종 동정\n\n\n ○ 대상해충 : 최...,24
1,1차년도\n1) Microarray를 통한 선천적 TRAIL 내성 표적 후보 유전자...,0
2,* 식물계자원 정련 및 최적 신서란 파이버 기초연구 개발\n\n\n- Tencel/...,0
3,# 소화기 암 진단용 분자영상 형광프로브 개발\n\n\n- 국소 도포형 소화기 암 ...,0
4,-In situ hybridization 검사의 정확성을 확인하기 위해 위암세포주 ...,0


In [11]:
test.head()

,요약문_연구내용
0,○ 1차년도\n\n . 개발 탐촉 시스템의 성능 평가 위한 표준 시편 제작 시...
1,연구과제1. 무한입자계의 동역학 / 작용소(operator) 방정식에 대한 연구\n...
2,본 연구는 퇴원 후 저출생체중아의 퇴원 후 추후관리 프로그램 중 가정방문 모델과 가...
3,(1) 0.5~1㎛의 가공정밀도(면조도) 구현\n\n\n - 국내에서는 연삭기를 제...
4,"총 연구기간은 3년으로, 연차별 연구내용 및 범위는 다음과 같다.\n\n ∙ 1차..."


In [12]:
#1. re.sub 한글 및 공백을 제외한 문자 제거
#2. okt 객체를 활용해 형태소 단위로 나눔
#3. remove_stopwords로 불용어 제거 
def preprocessing(text, okt, remove_stopwords=False, stop_words=[]):
    text=re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ]","", text)
    # word_text_norm=okt.normalize(text)
    word_text=okt.nouns(text)
    if remove_stopwords:
        word_review=[token for token in word_text if not token in stop_words]
    return word_review

In [18]:
stop_words=['은','는','이','가', '하','아','것','들','의','있','되','수','보','주','등','한', '및']
okt=Okt()
clean_train_text=[]
clean_test_text=[]

SystemError: java.lang.UnsatisfiedLinkError: Native Library C:\Anaconda3\Lib\site-packages\_jpype.cp38-win_amd64.pyd already loaded in another classloader